In [2]:
import os

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import minmax_scale

In [81]:
def load_data(file_name: str):

    pickle_file = f"{file_name}.pkl"
    parquet_file = f"{file_name}"

    if os.path.exists(pickle_file):
        print(f"Loading data from pickle file: {pickle_file}")
        df = pd.read_pickle(pickle_file)
    else:
        print(f"Pickle file not found. Loading data from parquet file: {parquet_file}")
        if not os.path.exists(parquet_file):
            raise FileNotFoundError(f"The parquet file {parquet_file} does not exist.")
        df = pd.read_parquet(parquet_file)
        df.to_pickle(pickle_file)
        print(f"Data saved to pickle file: {pickle_file}")
    return df

def save_data(df,file_name: str):

    pickle_file = f"{file_name}.pkl"

    print(f"Saving data to pickle file: {pickle_file}")
    df.to_pickle(pickle_file)
    print(f"Data saved to pickle file: {pickle_file}")
    return df

def analyse_data(df):

    num_unique_values = df.nunique()
    non_null_percentage = df.notnull().mean() * 100
    non_null_count = df.count()
    column_types = df.dtypes

    analysis_df = pd.DataFrame({
        'count_non_null_values': non_null_count,    
        '%_non_null_values': non_null_percentage,
        'nb_unique_values': num_unique_values,
        'column_type': column_types,
        })
    return analysis_df

def load_csv_data(file_name: str):

    pickle_file = f"{file_name}.pkl"
    parquet_file = f"{file_name}"

    if os.path.exists(pickle_file):
        print(f"Loading data from pickle file: {pickle_file}")
        df = pd.read_pickle(pickle_file)
    else:
        print(f"Pickle file not found. Loading data from csv file: {parquet_file}")
        if not os.path.exists(parquet_file):
            raise FileNotFoundError(f"The csv file {parquet_file} does not exist.")
        df = pd.read_csv(parquet_file)
        df.to_pickle(pickle_file)
        print(f"Data saved to pickle file: {pickle_file}")
    return df

In [4]:
df_client = load_data("/home/jonathan/Documents/OSP/osp_fil_rouge/Cours/etude_de_cas_entreprise/Rexel/clients_data_anonymized.parquet")
df_google_analysis = load_data("/home/jonathan/Documents/OSP/osp_fil_rouge/Cours/etude_de_cas_entreprise/Rexel/google_analytics_data_anonymized.parquet")
df_product = load_data("/home/jonathan/Documents/OSP/osp_fil_rouge/Cours/etude_de_cas_entreprise/Rexel/products_data_anonymized.parquet")
df_transaction = load_data("/home/jonathan/Documents/OSP/osp_fil_rouge/Cours/etude_de_cas_entreprise/Rexel/transactions_data_anonymized.parquet")

df_transaction

Loading data from pickle file: /home/jonathan/Documents/OSP/osp_fil_rouge/Cours/etude_de_cas_entreprise/Rexel/clients_data_anonymized.parquet.pkl
Loading data from pickle file: /home/jonathan/Documents/OSP/osp_fil_rouge/Cours/etude_de_cas_entreprise/Rexel/google_analytics_data_anonymized.parquet.pkl
Loading data from pickle file: /home/jonathan/Documents/OSP/osp_fil_rouge/Cours/etude_de_cas_entreprise/Rexel/products_data_anonymized.parquet.pkl
Loading data from pickle file: /home/jonathan/Documents/OSP/osp_fil_rouge/Cours/etude_de_cas_entreprise/Rexel/transactions_data_anonymized.parquet.pkl


,order_id,date,client_id,product_id,channel,quantity
0,ed81f77484,31,d8f195,a0c3d90c,TEL,2
1,59e4eb1ca8,3,8aea98,da91e60e,FACE,5
2,a0e0b229c7,27,9b2f5e,820ae967,DIGIT,6
3,a0e0b229c7,27,9b2f5e,820ae967,DIGIT,8
4,ec899f4fed,14,0ffc6e,cb721d76,TEL,1
...,...,...,...,...,...,...
467688,5922cc952f,28,e868ec,f1b05314,TEL,2
467689,5922cc952f,28,e868ec,d8e75c4d,TEL,2
467690,5922cc952f,28,e868ec,2533558b,TEL,2
467691,a01dd576a8,24,5bcb58,b4a89524,FACE,2


In [24]:
df_client

,client_id,type,market,agence_code
0,1feeca,Utilisateurs,Tertiaire,7386
1,72f291,Utilisateurs,Tertiaire,2ef3
2,697de3,Installateurs,Tertiaire,92ae
3,c03df6,Installateurs,Tertiaire,3dd3
4,f4e9c8,Utilisateurs,Industrie,5b66
...,...,...,...,...
29442,2a1ad3,Installateurs,Tertiaire,693c
29443,9b77ab,Installateurs,Habitat / Petit Tertiaire,dc8c
29444,27dfd0,Installateurs,Habitat / Petit Tertiaire,d6f5
29445,e72565,Installateurs,Industrie,95ba


In [66]:
df_unique_client = df_client.drop_duplicates(subset=['client_id']).reindex()  # Garde uniquement la première occurrence
#df_unique_client = df_unique_client.set_index('client_id')

df_unique_client#.count()

,client_id,type,market,agence_code
0,1feeca,Utilisateurs,Tertiaire,7386
1,72f291,Utilisateurs,Tertiaire,2ef3
2,697de3,Installateurs,Tertiaire,92ae
3,c03df6,Installateurs,Tertiaire,3dd3
4,f4e9c8,Utilisateurs,Industrie,5b66
...,...,...,...,...
29442,2a1ad3,Installateurs,Tertiaire,693c
29443,9b77ab,Installateurs,Habitat / Petit Tertiaire,dc8c
29444,27dfd0,Installateurs,Habitat / Petit Tertiaire,d6f5
29445,e72565,Installateurs,Industrie,95ba


In [67]:
df_client_analyze = analyse_data(df_unique_client)

df_client_analyze

,count_non_null_values,%_non_null_values,nb_unique_values,column_type
client_id,26024,100.0,26024,object
type,26024,100.0,7,object
market,26024,100.0,6,object
agence_code,26024,100.0,469,object


In [68]:
df_check = df_unique_client["agence_code"].drop_duplicates()

df_check.sample(6)

2134    30fe
545     1c65
529     94a3
1145    7653
365     5633
1946    ce8e
Name: agence_code, dtype: object

In [69]:
df_google_analysis_analyze = analyse_data(df_google_analysis)

df_google_analysis_analyze

,count_non_null_values,%_non_null_values,nb_unique_values,column_type
query_id,5662375,100.0,621048,object
date,5662375,100.0,31,int64
client_id,5662375,100.0,23662,object
page,5662375,100.0,172,int64
searchkeyword,5662375,100.0,225836,object
product_id,5662375,100.0,81116,object
is_seen,5662375,100.0,1,int64
nb_interactions_type1,5662375,100.0,8,int64
nb_interactions_type2,5662375,100.0,9,int64
interaction_type1,5662375,100.0,2,int64


In [14]:

df_google_analysis['searchkeyword_occurence'] = df_google_analysis['searchkeyword'].map(df_google_analysis['searchkeyword'].value_counts())

df_google_analysis


,query_id,date,client_id,page,searchkeyword,product_id,is_seen,nb_interactions_type1,nb_interactions_type2,interaction_type1,interaction_type2,is_interaction,position,searchkeyword_occurence
0,9ae42bb0ca,1,790218,1,hublot avec allumage automatique,0b7a94f0,1,0,0,0,0,0,1,8
1,9ae42bb0ca,1,790218,1,hublot avec allumage automatique,963872dd,1,0,0,0,0,0,2,8
2,9ae42bb0ca,1,790218,1,hublot avec allumage automatique,a85f381a,1,0,0,0,0,0,3,8
3,9ae42bb0ca,1,790218,1,hublot avec allumage automatique,7b5c8b46,1,0,0,0,0,0,4,8
4,9ae42bb0ca,1,790218,1,hublot avec allumage automatique,1085f20a,1,0,0,0,0,0,5,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5662370,cc5a590502,31,5fef66,1,bts e10-ds5,b87fa6e1,1,0,0,0,0,0,3,7
5662371,cc5a590502,31,5fef66,1,bts e10-ds5,d99010e8,1,0,0,0,0,0,4,7
5662372,cc5a590502,31,5fef66,1,bts e10-ds5,706d5d0f,1,0,0,0,0,0,5,7
5662373,cc5a590502,31,5fef66,1,bts e10-ds5,3027c1dc,1,0,0,0,0,0,6,7


In [15]:
# Plot the occurrences
occurrences = df_google_analysis[['searchkeyword', 'searchkeyword_occurence']].drop_duplicates()

occurrences = occurrences.sort_values(by='searchkeyword_occurence', ascending=False).reset_index(drop=True)

occurrences.head(30)

occurrences.sample(30)

,searchkeyword,searchkeyword_occurence
201192,pliograph plus 33-180,6
63290,cosse alu cuivre 185-8,14
193287,heavy duty poteau ou potelet,6
75269,disjoncteur mono si 32a,12
110856,cadre de finition 4 postes,9
72343,3g4 rigide,12
50108,ptt 99 28 paires 8/10,18
96,syt,2833
195131,gaine ict verte diametre 40,6
169847,pithon a bascule,7


In [16]:
df_google_analysis_tmp = df_google_analysis[df_google_analysis["client_id"] == '790218']

df_google_analysis_tmp.head(30)

,query_id,date,client_id,page,searchkeyword,product_id,is_seen,nb_interactions_type1,nb_interactions_type2,interaction_type1,interaction_type2,is_interaction,position,searchkeyword_occurence
0,9ae42bb0ca,1,790218,1,hublot avec allumage automatique,0b7a94f0,1,0,0,0,0,0,1,8
1,9ae42bb0ca,1,790218,1,hublot avec allumage automatique,963872dd,1,0,0,0,0,0,2,8
2,9ae42bb0ca,1,790218,1,hublot avec allumage automatique,a85f381a,1,0,0,0,0,0,3,8
3,9ae42bb0ca,1,790218,1,hublot avec allumage automatique,7b5c8b46,1,0,0,0,0,0,4,8
4,9ae42bb0ca,1,790218,1,hublot avec allumage automatique,1085f20a,1,0,0,0,0,0,5,8
5,9ae42bb0ca,1,790218,1,hublot avec allumage automatique,14277b75,1,0,0,0,0,0,6,8
6,9ae42bb0ca,1,790218,1,hublot avec allumage automatique,4357582b,1,0,0,0,0,0,7,8
7,9ae42bb0ca,1,790218,1,hublot avec allumage automatique,24e63570,1,0,0,0,0,0,8,8
554217,1bef1e2b73,4,790218,1,ho7vk,da227ecd,1,0,0,0,0,0,1,506
554218,1bef1e2b73,4,790218,1,ho7vk,c294d3ce,1,0,0,0,0,0,2,506


In [70]:
df_unique_client_count = df_unique_client

df_unique_client_count['client_id_occ'] = df_unique_client['client_id'].map(df_unique_client['client_id'].value_counts())

df_unique_client_count[df_unique_client["client_id_occ"] >1].sort_values(by='client_id', ascending=False).reset_index(drop=True)

,client_id,type,market,agence_code,client_id_occ


In [18]:
df_client_filter = df_unique_client[["type","market"]]

df_client_filter['type_market'] = df_client_filter['type'] + "_" + df_client_filter['market']

df_client_filter['type_market_occurence'] = df_client_filter["type_market"].map(df_client_filter["type_market"].value_counts())

df_client_filter.drop_duplicates()
#df_client_filter["market"].drop_duplicates()

/tmp/ipykernel_433121/3566787074.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_client_filter['type_market'] = df_client_filter['type'] + "_" + df_client_filter['market']
/tmp/ipykernel_433121/3566787074.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_client_filter['type_market_occurence'] = df_client_filter["type_market"].map(df_client_filter["type_market"].value_counts())


,type,market,type_market,type_market_occurence
0,Utilisateurs,Tertiaire,Utilisateurs_Tertiaire,2319
2,Installateurs,Tertiaire,Installateurs_Tertiaire,3573
4,Utilisateurs,Industrie,Utilisateurs_Industrie,3015
5,Intégrateurs,Industrie,Intégrateurs_Industrie,1301
6,Installateurs,Habitat / Petit Tertiaire,Installateurs_Habitat / Petit Tertiaire,13036
33,Intégrateurs,Habitat / Petit Tertiaire,Intégrateurs_Habitat / Petit Tertiaire,80
35,Intégrateurs,Tertiaire,Intégrateurs_Tertiaire,260
61,Installateurs,Industrie,Installateurs_Industrie,1532
88,Revendeurs,Autres,Revendeurs_Autres,206
212,Installateurs électriciens,Installateur,Installateurs électriciens_Installateur,25


In [76]:
df_client_google_merge = pd.merge(left=df_google_analysis,right=df_unique_client,how='left',on="client_id")

df_google_analysis

,query_id,date,client_id,page,searchkeyword,product_id,is_seen,nb_interactions_type1,nb_interactions_type2,interaction_type1,interaction_type2,is_interaction,position,searchkeyword_occurence
0,9ae42bb0ca,1,790218,1,hublot avec allumage automatique,0b7a94f0,1,0,0,0,0,0,1,8
1,9ae42bb0ca,1,790218,1,hublot avec allumage automatique,963872dd,1,0,0,0,0,0,2,8
2,9ae42bb0ca,1,790218,1,hublot avec allumage automatique,a85f381a,1,0,0,0,0,0,3,8
3,9ae42bb0ca,1,790218,1,hublot avec allumage automatique,7b5c8b46,1,0,0,0,0,0,4,8
4,9ae42bb0ca,1,790218,1,hublot avec allumage automatique,1085f20a,1,0,0,0,0,0,5,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5662370,cc5a590502,31,5fef66,1,bts e10-ds5,b87fa6e1,1,0,0,0,0,0,3,7
5662371,cc5a590502,31,5fef66,1,bts e10-ds5,d99010e8,1,0,0,0,0,0,4,7
5662372,cc5a590502,31,5fef66,1,bts e10-ds5,706d5d0f,1,0,0,0,0,0,5,7
5662373,cc5a590502,31,5fef66,1,bts e10-ds5,3027c1dc,1,0,0,0,0,0,6,7


In [73]:
df_client_google_merge

,query_id,date,client_id,page,searchkeyword,product_id,is_seen,nb_interactions_type1,nb_interactions_type2,interaction_type1,interaction_type2,is_interaction,position,searchkeyword_occurence,type,market,agence_code,client_id_occ
0,9ae42bb0ca,1,790218,1,hublot avec allumage automatique,0b7a94f0,1,0,0,0,0,0,1,8,Installateurs,Habitat / Petit Tertiaire,01fc,1
1,9ae42bb0ca,1,790218,1,hublot avec allumage automatique,963872dd,1,0,0,0,0,0,2,8,Installateurs,Habitat / Petit Tertiaire,01fc,1
2,9ae42bb0ca,1,790218,1,hublot avec allumage automatique,a85f381a,1,0,0,0,0,0,3,8,Installateurs,Habitat / Petit Tertiaire,01fc,1
3,9ae42bb0ca,1,790218,1,hublot avec allumage automatique,7b5c8b46,1,0,0,0,0,0,4,8,Installateurs,Habitat / Petit Tertiaire,01fc,1
4,9ae42bb0ca,1,790218,1,hublot avec allumage automatique,1085f20a,1,0,0,0,0,0,5,8,Installateurs,Habitat / Petit Tertiaire,01fc,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5662370,cc5a590502,31,5fef66,1,bts e10-ds5,b87fa6e1,1,0,0,0,0,0,3,7,Installateurs,Tertiaire,97f3,1
5662371,cc5a590502,31,5fef66,1,bts e10-ds5,d99010e8,1,0,0,0,0,0,4,7,Installateurs,Tertiaire,97f3,1
5662372,cc5a590502,31,5fef66,1,bts e10-ds5,706d5d0f,1,0,0,0,0,0,5,7,Installateurs,Tertiaire,97f3,1
5662373,cc5a590502,31,5fef66,1,bts e10-ds5,3027c1dc,1,0,0,0,0,0,6,7,Installateurs,Tertiaire,97f3,1


In [82]:
df_product_transaction_lea = load_csv_data("client_metrics_Léa.csv")

df_product_transaction_lea

Pickle file not found. Loading data from csv file: client_metrics_Léa.csv
Data saved to pickle file: client_metrics_Léa.csv.pkl


,client_id,num_orders,num_products,product_id_x,product_id_y,channel,num_channels,avg_quantity_per_order,brand_name,favorite_prd_category_1,favorite_prd_category_2,favorite_prd_category_3,favorite_prd_category_4
0,000195,24,43,98b22049,86c93da7,FACE,3,17.629032,Fils et Câbles Rexel,CABLES,CABLE ET FIL DOMESTIQUE,FIL RIGIDE H07VU,Fil rigide H07VU
1,0002b0,6,6,ed84e264,248d64ac,FACE,1,2.250000,Iboco,APPAREILLAGE ET CONTROLE DU BATIMENT,BOITE D'ENCASTREMENT ET DE DERIVATION,BOITE ETANCHE,Boîte étanche
2,000973,7,15,58fbdd34,d2812325,TEL,1,34.750000,Schneider Electric,"OUTILLAGE, MESURE ET FIXATION",CABLE INDUSTRIEL,CABLE SOUPLE H07RNF ET RV-K,FIXATION TUBE ET CABLE PLASTIQUE
3,0013cd,2,3,de22d488,46eb79a3,DIGIT,1,34.666667,Fils et Câbles Rexel,CABLES,CABLE INDUSTRIEL,CABLE R2V SENSIBLE 1.5² ET 2.5²,CABLE R2V SENSIBLE 1.5² ET 2.5²
4,0017ab,6,3,67c69581,81d28c36,FACE,2,18.230769,Bizline,"OUTILLAGE, MESURE ET FIXATION",MARQUAGE ET REPERAGE,COLLIER DE CABLAGE,COLLIER DE CABLAGE
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26019,fff808,2,4,af4a90b7,767167eb,DIGIT,1,26.000000,Cembre,PROCESS INDUSTRIEL,CONNECTIQUE ET RACCORDEMENT,EMBOUT DE CABLAGE,Embout de câblage
26020,fff8e3,6,7,4fbaa14b,208a11c9,DIGIT,2,1.142857,Schneider Electric,PROCESS INDUSTRIEL,COMMANDE ET PROTECTION MOTEUR,SECTIONNEUR,Sectionneur
26021,fffb11,3,6,7ee35443,2743b7f5,FACE,1,42.666667,Spit,"OUTILLAGE, MESURE ET FIXATION",FIXATION,VISSERIE BOULONNERIE CHEVILLES,Visserie Boulonnerie Chevilles
26022,fffc3d,2,4,25782c8a,25782c8a,DIGIT,2,1.000000,Schneider Electric,APPAREILLAGE ET CONTROLE DU BATIMENT,APPAREILLAGE MURAL,APPAREILLAGE ETANCHE IP>=55,APPAREILLAGE ETANCHE IP>=55


In [95]:
df_product_transaction_lea_update = load_csv_data("client_metrics_Léa_update.csv")

df_product_transaction_lea_update

Pickle file not found. Loading data from csv file: client_metrics_Léa_update.csv
Data saved to pickle file: client_metrics_Léa_update.csv.pkl


,client_id,num_orders,num_products,product_id_x,product_id_y,channel,num_channels,avg_quantity_per_order,brand_name,favorite_prd_category_1,favorite_prd_category_2,favorite_prd_category_3,favorite_prd_category_4,second_top_product,third_top_product
0,000195,24,43,98b22049,86c93da7,FACE,3,17.629032,Fils et Câbles Rexel,CABLES,CABLE ET FIL DOMESTIQUE,FIL RIGIDE H07VU,Fil rigide H07VU,11d34252,2d8c77f3
1,0002b0,6,6,ed84e264,248d64ac,FACE,1,2.250000,Iboco,APPAREILLAGE ET CONTROLE DU BATIMENT,BOITE D'ENCASTREMENT ET DE DERIVATION,BOITE ETANCHE,Boîte étanche,644e50d8,6de087d2
2,000973,7,15,58fbdd34,d2812325,TEL,1,34.750000,Schneider Electric,"OUTILLAGE, MESURE ET FIXATION",CABLE INDUSTRIEL,CABLE SOUPLE H07RNF ET RV-K,FIXATION TUBE ET CABLE PLASTIQUE,06f44771,0b2f35fd
3,0013cd,2,3,de22d488,46eb79a3,DIGIT,1,34.666667,Fils et Câbles Rexel,CABLES,CABLE INDUSTRIEL,CABLE R2V SENSIBLE 1.5² ET 2.5²,CABLE R2V SENSIBLE 1.5² ET 2.5²,b17d132e,de22d488
4,0017ab,6,3,67c69581,81d28c36,FACE,2,18.230769,Bizline,"OUTILLAGE, MESURE ET FIXATION",MARQUAGE ET REPERAGE,COLLIER DE CABLAGE,COLLIER DE CABLAGE,67c69581,8edf9167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26019,fff808,2,4,af4a90b7,767167eb,DIGIT,1,26.000000,Cembre,PROCESS INDUSTRIEL,CONNECTIQUE ET RACCORDEMENT,EMBOUT DE CABLAGE,Embout de câblage,af4a90b7,d1cff054
26020,fff8e3,6,7,4fbaa14b,208a11c9,DIGIT,2,1.142857,Schneider Electric,PROCESS INDUSTRIEL,COMMANDE ET PROTECTION MOTEUR,SECTIONNEUR,Sectionneur,4fbaa14b,62e90d5c
26021,fffb11,3,6,7ee35443,2743b7f5,FACE,1,42.666667,Spit,"OUTILLAGE, MESURE ET FIXATION",FIXATION,VISSERIE BOULONNERIE CHEVILLES,Visserie Boulonnerie Chevilles,3dd058d6,7e608a67
26022,fffc3d,2,4,25782c8a,25782c8a,DIGIT,2,1.000000,Schneider Electric,APPAREILLAGE ET CONTROLE DU BATIMENT,APPAREILLAGE MURAL,APPAREILLAGE ETANCHE IP>=55,APPAREILLAGE ETANCHE IP>=55,3a2c84e1,a86aa7d4


In [96]:
df_client_prod_trans_merge = pd.merge(left=df_unique_client,right=df_product_transaction_lea_update,how='inner',on="client_id")

df_client_prod_trans_merge = df_client_prod_trans_merge.drop(columns=["client_id_occ"])
df_client_prod_trans_merge = df_client_prod_trans_merge.drop(columns=["counts"])

df_client_prod_trans_merge = df_client_prod_trans_merge.rename(columns={'num_orders': 'nb_total_orders',
                                                                        'num_products':'nb_different_product_ordered',
                                                                        'product_id_x':'most_buyed_product',
                                                                        'product_id_y':'most_ordered_product_1',
                                                                        'second_top_product': 'most_ordered_product_2',
                                                                        'third_top_product' : 'most_ordered_product_3',
                                                                        'channel':'preferred_channel',
                                                                        'num_channels':'nb_different_channels',
                                                                        })

df_client_prod_trans_merge["avg_quantity_per_order"] = df_client_prod_trans_merge["avg_quantity_per_order"].round().astype(int)

In [97]:
df_client_prod_trans_merge

,client_id,type,market,agence_code,nb_total_orders,nb_different_product_ordered,most_buyed_product,most_ordered_product_1,preferred_channel,nb_different_channels,avg_quantity_per_order,brand_name,favorite_prd_category_1,favorite_prd_category_2,favorite_prd_category_3,favorite_prd_category_4,most_ordered_product_2,most_ordered_product_3
0,1feeca,Utilisateurs,Tertiaire,7386,2,2,8cd3b1ff,8cd3b1ff,TEL,1,21,Somfy,SECURITE ET COMMUNICATION,AUTOMATISME D'OUVERTURE,MOTORISATION VOLET,Motorisation volet,deb53268,NaN
1,72f291,Utilisateurs,Tertiaire,2ef3,1,2,65a12773,65a12773,DIGIT,1,1,Energizer france,"OUTILLAGE, MESURE ET FIXATION",EQUIPEMENT GENERAL,PILE,PILE,c0d77c8b,NaN
2,697de3,Installateurs,Tertiaire,92ae,12,18,c0fa44c7,32b8b06d,DIGIT,3,95,Fils et Câbles Rexel,CABLES,CABLE INDUSTRIEL,CABLE AR2V ET ALUMINIUM,CABLE AR2V ET ALUMINIUM,3d077f06,0e080d59
3,c03df6,Installateurs,Tertiaire,3dd3,7,14,56038467,044f2ce4,DIGIT,3,20,Courant,CONDUITS,CONDUIT ROND,CONDUIT ICTA GRIS ET GL,CONDUIT ICTA GRIS ET GL,0ee6a4a0,42944aa9
4,f4e9c8,Utilisateurs,Industrie,5b66,6,19,cdcccb6a,0040f4a8,DIGIT,1,17,Entrelec,PROCESS INDUSTRIEL,CONNECTIQUE ET RACCORDEMENT,BLOC DE JONCTION,Bloc de jonction,1c5fcb11,1fcde5cf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26019,2a1ad3,Installateurs,Tertiaire,693c,38,57,dee353ee,5b426ff3,TEL,3,37,Legrand,"OUTILLAGE, MESURE ET FIXATION",MARQUAGE ET REPERAGE,REPERE POUR FILS ET CABLES,REPERE POUR FILS ET CABLES,e2f53fb8,0553d1a7
26020,9b77ab,Installateurs,Habitat / Petit Tertiaire,dc8c,4,8,5c66d6ea,e2e53d04,FACE,1,12,Elydan,CONDUITS,CONDUIT ROND,CONDUIT ICTA,Conduit ICTA,04d5b03d,4433100a
26021,27dfd0,Installateurs,Habitat / Petit Tertiaire,d6f5,9,21,3c170f11,168a92fa,DIGIT,1,17,Blm distribution,"OUTILLAGE, MESURE ET FIXATION",CONNEXION ET ACCESSOIRES D'INSTALLATION ELECTR...,BARRETTE ET BORNE SANS VIS,BARRETTE ET BORNE SANS VIS,20232a8f,283dcdf0
26022,e72565,Installateurs,Industrie,95ba,2,3,5488fd91,141f7ff6,DIGIT,1,5,3M france,"OUTILLAGE, MESURE ET FIXATION",MARQUAGE ET REPERAGE,MANCHON ET GAINE THERMORETRACTABLE,MANCHON ET GAINE THERMORETRACTABLE,5488fd91,deb53268


In [94]:
df_product

,product_id,prd_long_description,prd_short_description,prd_title,brand_code,brand_name,prd_category_1,prd_category_2,prd_category_3,prd_category_4,has_image,prd_classification
0,a5f0e428,Cl-d<br/><br/>Cosses colorées à deux trous et ...,Cosse non isolée code couleur rouge 350 MCM 2 ...,Cosse non isolée code couleur rouge 350 MCM 2 ...,CMB,Cembre,PROCESS INDUSTRIEL,CONNECTIQUE ET RACCORDEMENT,RACCORD ET COSSE,"Raccord, Cosse",0.0,Cosse tubulaire pour conducteur en cuivre
1,6eb6d804,Application : > Permet le raccordement de 1 à ...,Collecteur raccord d'étage galvanisé - Diamètr...,Collecteur raccord d'étage galvanisé - Diamètr...,ALD,Aldes,"CHAUFFAGE ELEC, CLIM ET VENTILATION",VENTILATION,CONDUIT DE VENTILATION ET ACCESSOIRES,CONDUIT DE VENTILATION ET ACCESSOIRES,1.0,"Raccord T-Y, gaine de ventilation ronde"
2,7f11dead,La boîte Mini-Com® de fixation en saillie peut...,"Surface Mount Box, 4 Port, Off White","Surface Mount Box, 4 Port, Off White",PUI,Panduit Solutions Réseaux,INFRASTRUCTURE ET RESEAU,PRODUIT CUIVRE,CONNECTEUR ET PRISE AUTRE POUR RESEAU,Connecteur et Prise autre pour réseau,1.0,Boîtier en saillie pour de l'appareillage
3,a0e11db0,Rcc,RCC 500/450 - réduction conique concentrique d...,RCC 500/450 - réduction conique concentrique d...,ELG,Atlantic Climatisation & Ventilation,"CHAUFFAGE ELEC, CLIM ET VENTILATION",VENTILATION,CONDUIT DE VENTILATION ET ACCESSOIRES,CONDUIT DE VENTILATION ET ACCESSOIRES,1.0,"Cône d'aspiration, gaine de ventilation ronde"
4,55c32fba,"Luminaire profilé LED de grande qualité, en Hy...",pure3 green suspension noir sat. 4222x70mmLED ...,pure3 green suspension noir sat. 4222x70mm LED...,AH4,Planlicht,ECLAIRAGE,LUMINAIRE D'INTERIEUR DECORATIF,SUSPENSION D'ECLAIRAGE DECORATIF,SUSPENSION D'ECLAIRAGE DECORATIF LED INTEGREE,1.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...
127518,4315b34c,Finition mécanisme : noir Livré avec anneau de...,Disq 60 Incara avec chargeur USB TypeA+TypeC 1...,Disq60 Incara chargeur USB A+C et cordon avec ...,LEG,Legrand,CONDUITS,ALIMENTATION ET EQUIPEMENT DU POSTE DE TRAVAIL,NOURRICE ET EQUIPEMENT DE BUREAU,NOURRICE ET EQUIPEMENT DE BUREAU,1.0,Multiprise de table
127519,7e119064,Chaleur douce Jusqu'à 30% d'économies d'énergi...,Radiateur chaleur douce - Baléares 2 horizonta...,Radiateur chaleur douce Baléares 2 horizontal ...,EET,Thermor,"CHAUFFAGE ELEC, CLIM ET VENTILATION",CHAUFFAGE DOMESTIQUE,RADIATEUR ELECTRIQUE CALOPORTEUR,RADIATEUR ELECTRIQUE CALOPORTEUR,1.0,Radiateur (électrique)
127520,fd22598a,None,SIMOTICS SD type de moteur: 1CV3182B moteur à ...,"18,5 kW 400V'/690VY 4p B3 fonte P IE3",I2N,Innomotics,PROCESS INDUSTRIEL,ACTIONNEUR ELECTRIQUE,MOTEUR ELECTRIQUE,Moteur électrique,0.0,Moteur électrique
127521,a20e87f1,Pour toutes applications industrielles et tert...,Dalle perforée BS à bords roulés fermés de séc...,Dalle perforée BS à bords roulés fermés de séc...,NDX,Niedax France,CONDUITS,CHEMIN DE CABLE ET ECHELLE,CHEMIN DE CABLE TOLE,CHEMIN DE CABLE TOLE,1.0,Chemin de câble / chemin de câble grande portée


In [99]:
df_client_prod_trans_merge.to_csv("client_transaction_product_file.csv")

In [101]:
df_client

,client_id,type,market,agence_code
0,1feeca,Utilisateurs,Tertiaire,7386
1,72f291,Utilisateurs,Tertiaire,2ef3
2,697de3,Installateurs,Tertiaire,92ae
3,c03df6,Installateurs,Tertiaire,3dd3
4,f4e9c8,Utilisateurs,Industrie,5b66
...,...,...,...,...
29442,2a1ad3,Installateurs,Tertiaire,693c
29443,9b77ab,Installateurs,Habitat / Petit Tertiaire,dc8c
29444,27dfd0,Installateurs,Habitat / Petit Tertiaire,d6f5
29445,e72565,Installateurs,Industrie,95ba
